In [1]:
import requests
import pandas as pd

import sys
sys.path.append('..')
import folium
from folium.features import *
from folium.element import *
from folium.plugins import *


In [2]:
data = requests.get('https://raw.githubusercontent.com/austinlyons/dcjs-leaflet-untappd/master/untappd.json')\
    .json()['response']['beers']['items']
data = pd.DataFrame(data)

In [3]:
data['beer_rating_score']=data['beer'].apply(lambda x: x['rating_score'])
data['beer_beer_abv'] = data['beer'].apply(lambda x: x['beer_abv'])
data['beer_beer_ibu'] = data['beer'].apply(lambda x: x['beer_ibu'])
data['beer_beer_style'] = data['beer'].apply(lambda x: x['beer_style'])
data['beer_beer_name'] = data['beer'].apply(lambda x: x['beer_name'])
data['brewery_brewery_name'] = data['brewery'].apply(lambda x: x['brewery_name'])
data['dt']  = data.first_had.apply(pd.Timestamp)
data['first_had_year'] = data.dt.apply(lambda x: x.strftime('%Y'))#str(x.year))
data['first_had_month'] = data.dt.apply(lambda x: x.strftime('%b'))
data['first_had_day'] = data.dt.apply(lambda x: x.strftime('%a'))

data['popup'] = data.apply(lambda x: "<p>" + x['brewery_brewery_name'] + " "\
      + x['brewery']['location']['brewery_city'] + " "\
      + x['brewery']['location']['brewery_state'] + "</p>", axis=1)

data['lat'] = data['brewery'].apply(lambda x: x['location']['lat'])
data['lng'] = data['brewery'].apply(lambda x: x['location']['lng'])

data.drop(['beer','brewery', 'first_checkin_id', 'first_had', 'recent_checkin_id'],axis=1, inplace=True)

In [4]:
data.head()

,count,rating_score,beer_rating_score,beer_beer_abv,beer_beer_ibu,beer_beer_style,beer_beer_name,brewery_brewery_name,dt,first_had_year,first_had_month,first_had_day,popup,lat,lng
0,1,4.25,3.677,5.3,27,Scotch Ale / Wee Heavy,90 Shilling,Odell Brewing Co.,2015-07-10 19:23:07-05:00,2015,Jul,Fri,<p>Odell Brewing Co. Fort Collins CO</p>,40.5888,-105.0620
1,1,4.00,3.556,5.8,0,Witbier,Naked Nun,Adelbert's Brewery,2015-07-08 20:16:02-05:00,2015,Jul,Wed,<p>Adelbert's Brewery Austin TX</p>,30.3827,-97.7202
2,1,4.00,3.501,5.2,30,White IPA,(512) White IPA,(512) Brewing Company,2015-07-06 21:44:07-05:00,2015,Jul,Mon,<p>(512) Brewing Company Austin TX</p>,30.2236,-97.7697
3,1,4.25,3.851,5.4,14,Hefeweizen,Weihenstephaner Hefeweissbier,Bayerische Staatsbrauerei Weihenstephan,2015-07-03 12:27:34-05:00,2015,Jul,Fri,<p>Bayerische Staatsbrauerei Weihenstephan Fre...,48.3952,11.7289
4,1,3.75,3.611,6.6,62,American IPA,Lost Gold IPA,Real Ale Brewing Company,2015-06-27 18:26:51-05:00,2015,Jun,Sat,<p>Real Ale Brewing Company Blanco TX</p>,30.1133,-98.4128


In [5]:
param = {
      "dataCount" : [
          {
              "name" : "DC1",
              "divId" : "#data-count"
              },
          {
              "name" : "DC2",
              "divId" : "#data-count2"
              }
          ],
      "pieChart" : [
          {
              "name" : "Year",
              "column" : "first_had_year",
              "divId" : "#chart-ring-year",
              "width" : 150,
              "height": 150,
              "innerRadius" : 20,
              "order" : [2013, 2014, 2015, 2016]
              },
          {
              "name" : "Weekday",
              "column" : "first_had_day",
              "divId" : "#chart-ring-day",
              "width" : 150,
              "height": 150,
              "innerRadius" : 20,
              "order" : ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
              },
          {
              "name" : "Month",
              "column" : "first_had_month",
              "divId" : "#chart-ring-month",
              "width" : 150,
              "height": 150,
              "innerRadius" : 20,
              "order" : [
                  'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
              }
          ],
      "barChart" : [
          {
              "name" : "Rating",
              "column" : "rating_score",
              "divId" : "#chart-rating-count",
              "width" : 300,
              "height" : 180,
              "domain" : [0.,5.2],
              "groupby" : 0.25,
              "xAxisLabel" : "My rating",
              "yAxisLabel" : "Count",
              "margins" : {'top': 10, 'right': 20, 'bottom': 50, 'left': 50},
              "xAxisTickValues" : [0, 1, 2, 3, 4, 5],
              "timeFormat" : None,
              "barPadding" : 0.1
              },
          {
              "name" : "Community rating",
              "column" : "beer_rating_score",
              "divId" : "#chart-community-rating-count",
              "width" : 300,
              "height" : 180,
              "domain" : [0.3,5.3],
              "groupby" : 1,
              "xAxisLabel" : "Community rating",
              "yAxisLabel" : "Count",
              "margins" : {'top': 10, 'right': 20, 'bottom': 50, 'left': 50},
              "xAxisTickValues" : [0, 1, 2, 3, 4, 5],
              "timeFormat" : None,
              "barPadding" : 0.01
              },
          {
              "name" : "Alcohol By Volume (%)",
              "column" : "beer_beer_abv",
              "divId" : "#chart-abv-count",
              "width" : 300,
              "height" : 180,
              "domain" : [0.,13],
              "groupby" : 0.5,
              "xAxisLabel" : "Alcohol By Volume (%)",
              "yAxisLabel" : "Count",
              "margins" : {'top': 10, 'right': 20, 'bottom': 50, 'left': 50},
              "xAxisTickValues" : [0, 2, 4, 6, 8, 10, 12],
              "barPadding" : 0.1
              },
          {
              "name" : "Datetime",
              "column" : "dt",
              "divId" : "#chart-dt-count",
              "width" : 600,
              "height" : 180,
              "domain" : [1356998400000,1451606400000],
              "groupby" : 86400000*30,
              "xAxisLabel" : "Datetime",
              "yAxisLabel" : "Count",
              "margins" : {'top': 10, 'right': 20, 'bottom': 50, 'left': 50},
              "xAxisTickValues" : [1356998400000, 1388534400000, 1420070400000, 1451606400000],
              "timeFormat" : "%b%Y",
              "barPadding" : 0.1
              },
          {
              "name" : "Datetime",
              "column" : "dt",
              "divId" : "#chart-dt2-count",
              "width" : 600,
              "height" : 180,
              "domain" : [1356998400000,1451606400000],
              "groupby" : 86400000*375,
              "xAxisLabel" : "Datetime",
              "yAxisLabel" : "Count",
              "margins" : {'top': 10, 'right': 20, 'bottom': 50, 'left': 50},
              "xAxisTickValues" : [1356998400000, 1420070400000],
              "timeFormat" : "%b%Y",
              "barPadding" : 0.1
              },
          {
              "name" : "International Bitterness Units",
              "column" : "beer_beer_ibu",
              "divId" : "#chart-ibu-count",
              "width" : 300,
              "height" : 180,
              "domain" : [0.,100],
              "groupby" : 5,
              "xAxisLabel" : "International Bitterness Units",
              "yAxisLabel" : "Count",
              "margins" : {'top': 10, 'right': 20, 'bottom': 50, 'left': 50},
              "xAxisTickValues" : [0, 20, 40, 60, 80, 100],
              "timeFormat" : None,
              "barPadding" : 0.1
              }
          ]
      }

In [6]:
#html = jinja2.Template(open('./dcjs-leaflet-untappd/index.html.tmpl').read())
#_repr_html_(html.render(data=pd.json.dumps(data.to_dict(orient='records')),
#                       param=pd.json.dumps(param)))

In [7]:
f = folium.element.Figure(height=1500)

c = Crossfilter(pd.json.dumps(data.to_dict(orient='records')),
                width="98%", left="2%",
                position='absolute')
f.add_children(c)

row0 = Div(class_="row", height=300, width="100%")
row1 = Div(class_="row", height=200, width="100%")
row2 = Div(class_="row", height=200, width="100%")
row3 = Div(class_="row", height=200, width="100%")
row4 = Div(class_="row", height=500, width="100%")
c.add_children(row0)
c.add_children(row1)
c.add_children(row2)
c.add_children(row3)
c.add_children(row4)

m = folium.Map()
m.add_children(FeatureGroupFilter(c))
row0.add_children(m)

for i, x in enumerate(param['pieChart']):
    row1.add_children(PieFilter(c, x['column'], name=x['name'],
                                width=150, height=150, inner_radius=20,
                 order=x['order'], class_="col-xs-4 pie-chart"))

for i, x in enumerate(param['barChart']):
    row = row2 if i<3 else row3
    row.add_children(BarFilter(
        c,
        x['column'],
        width=250,
        height=150,
        bar_padding=0.1,
        domain=x['domain'],
        groupby=x['groupby'],
        xlabel=x['xAxisLabel'],
        ylabel=x["yAxisLabel"],
        margins=x['margins'],
        xticks=x['xAxisTickValues'],
        time_format=x.get('timeFormat',None),
        class_="col-xs-4 dc-chart"))

row4.add_children(TableFilter(c,
                             ['brewery_brewery_name', 'beer_beer_name', 'beer_beer_style', 'rating_score',
                              'beer_rating_score', 'beer_beer_abv', 'beer_beer_ibu'
                             ],
                             class_="table table-bordered table-striped"))
    
f